# Accessing Art Museums

In [1]:
print('Hello World')

Hello World


In [1]:
#import libraries
import json
import pandas as pd
import numpy as np

# Load data
The following data sets are used in this project:
1. US Art Museums (CSV) - Art museums in the United States with website, city, state, zip code, and cost of admission
2. 2022 Census Income Data (CSV) - Median household income per zip code in the United States